## Notebook for generating plots for Figures 1 & 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from tqdm import tqdm
import numpy.linalg as nla
import seaborn as sns
import scipy as sp
import pandas as pd
import sympy as sym
from odeintw import odeintw
import multitype_new as mt
import importlib
importlib.reload(mt)
from scipy.integrate import odeint

cmp = plt.cm.get_cmap('Set2')
sns.set_palette("Set2")

In [68]:
# Parameters to change 

ntypes = 8
# Number of types-at-birth
nexposed = int(ntypes/2)


# Disease Parameters

gamma = 0.25 # Recovery Rate
sigma = 1/3  # Progression rate from exposed to infectious

beta_baseline = 0.675 #np.squeeze(pd.read_csv('VOC_beta.csv',header=None).to_numpy())
beta_vec = np.array((0.675, 0.75, 0.875, 0.95, 1.1))
Reff_vec = beta_vec / gamma
# print('Running for beta = ' + str(beta_baseline))

const_vec = np.squeeze(pd.read_csv('const_vec_in.csv',header=None).to_numpy()) #np.array((1., sus_ur, sus_vu, sus_vr))
prop_vec = np.squeeze(pd.read_csv('prop_vec_in.csv',header=None).to_numpy()) #np.array((p_sus*p_no_vac, p_rec*p_no_vac, p_sus*p_vac, p_rec*p_vac))
# const_vec = np.array((1, 0.4, 0.6, 0.3))
# prop_vec = np.array((0.4, 0.3, 0.1, 0.2))
# assert(np.sum(prop_vec) == 1)

scale = const_vec*prop_vec

epsilon = 1e-3
RV = beta_baseline/gamma

Tmax = 400 
num = Tmax 
time_vec = np.linspace(0, Tmax, num=num)
dtime = time_vec[1]-time_vec[0]



# Vector of immigrants at each type - currently only 1 immigrant of type 1 (unvaccinated, no prior infection) every 5 days

im = 0
im_vec = np.zeros(ntypes)
im_vec[0] = 0.1
y0 = np.zeros(ntypes)
y0[0] = 1
y0_im = np.zeros(ntypes)
y0_im[0] = 0
ics = np.zeros(ntypes)
ics_im = np.zeros(ntypes+1)
ics_im[-1] = 1

trans_rec = 1 - 0.4 # Reduction in transmission if recovered from previous strain
trans_vac = 1 - 0.7 # Reduction in transmission if vaccinated (but not previously infected)
trans_all = np.min((trans_rec, trans_vac)) # Reduction in transmission if vaccinated and recovered from previous strain

trans_vec = np.array((1., trans_rec, trans_vac, trans_all))
trans_vec = np.ones(nexposed)



In [ ]:
n_betas = len(beta_vec)
q_mat = np.zeros((n_betas, len(time_vec)))
rho_mat = np.zeros((n_betas, len(time_vec)))

R_vec = np.zeros(n_betas)


# Initialise Mean
mean = np.zeros((n_betas, len(time_vec)))
mean_im = np.zeros((n_betas, len(time_vec)))
plot_log = 1

mean_arr = np.zeros((n_betas, ntypes, len(time_vec)))
    
# Initialise Variance
variance = np.zeros((n_betas, len(time_vec)))
variance_im = np.zeros((n_betas, len(time_vec)))
lamb_vec = np.zeros(n_betas) # Vector of growth rates for each R0
var_mat = np.zeros((n_betas, len(time_vec), ntypes, ntypes))
# Initialise Tstar
Tstar_idxs = np.zeros(n_betas)
Tstars = np.zeros(n_betas)
epsilon = 1e-3
# Plotting Parameters
showplot = 100
showplot_rho = 300

R_effective_vec = np.zeros(n_betas)
for r, beta_baseline in enumerate(beta_vec):
    # beta_baseline = Reff*gamma

    RV = beta_baseline / gamma
    
    omega_vec = sigma * np.ones(ntypes) 

    
    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta_baseline)*trans_vec[nex] + gamma
        

    def P(u, t): # Offspring distribution

        pvec = np.zeros_like(u)
        nexposed = int(ntypes/2)
        omega = (np.sum(prop_vec * const_vec * beta_baseline) + gamma)
        

        sum_gen_funcs = np.sum((beta_baseline*prop_vec/omega)*const_vec*u[:nexposed])
         
        for nex in range(0, nexposed):

            pvec[nex] = u[nex+nexposed] 
            pvec[nex+nexposed] =  u[nex+nexposed] * sum_gen_funcs  + (gamma/omega)
        
        return pvec            
            
            
    eta = im_vec.astype(float)
    

# Mean of the Process
# Matrix of first partial derivatives of offspring distribution minus lifetimes*identity
    Omat = np.zeros((ntypes, ntypes))
    omega_vec = sigma * np.ones(ntypes)       

    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta_baseline)*trans_vec[nex] + gamma
        Omat[nex, nex+nexposed] = omega_vec[nex]
        Omat[nex+nexposed, :nexposed] = const_vec*prop_vec*beta_baseline *trans_vec[nex]
        Omat[nex+nexposed, nex+nexposed] = np.sum(const_vec * beta_baseline * prop_vec)*trans_vec[nex]

    Omat -= np.diag(omega_vec)


    

    Mvec = odeint(mt.set_mean_odes, y0, time_vec, args = (Omat,)).T
    mean_arr[r, :, :] = Mvec

    mean[r, :] = np.sum(Mvec, axis=0)

    Mvec_im = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec)).T
    mean_im[r, :] = np.sum(Mvec_im, axis=0)

    
    rho = (odeint(mt.set_odes, ics, time_vec, args = (P, omega_vec)).T )# Solve ODEs for probabilities of extinction/zero
    rho_im = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec)).T )[-1, :]
    q = np.ones_like(time_vec)
    for i in range(ntypes):
        q *= rho[i, :] ** y0[i]
    
    # Variance and Coefficient of Variation
    param_dict = {"a":scale[0], "b":scale[1], "c":scale[2], "d":scale[3], "s":sigma, "g":gamma, "beta":beta_baseline}

    variance[r, :] = mt.variance(time_vec, y0, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma])
    variance_im[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec)
 
    RV = np.round(Reff_vec[r], 2)
    sig_over_mean = (np.sqrt(variance[r, :])/(mean[r, :]))

    def Jacobian_mat(beta, gamma, sigma, ntypes = ntypes, const_vec = const_vec, prop_vec = prop_vec, trans_vec = trans_vec, omega_vec = omega_vec):
        Omat = np.zeros((ntypes, ntypes))
        omega_vec = sigma * np.ones(ntypes)       
        nexposed = int(ntypes/2)
        for nex in range(0, nexposed):
            omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta)*trans_vec[nex] + gamma
            Omat[nex, nex+nexposed] = omega_vec[nex]
            Omat[nex+nexposed, :nexposed] = const_vec*prop_vec*beta *trans_vec[nex]
            Omat[nex+nexposed, nex+nexposed] = np.sum(const_vec * beta * prop_vec)*trans_vec[nex]

        Omat -= np.diag(omega_vec)
        return Omat


    # Calculate R_eff from Next-Generation matrix (Diekmann 2010)
    Sigma_mat = Jacobian_mat(beta_baseline, gamma = gamma, sigma = sigma)
    Sigma_mat[nexposed:, :nexposed] = 0
    T_mat = Jacobian_mat(beta_baseline, gamma = gamma, sigma = sigma) - Sigma_mat
    next_gen = - T_mat @ nla.inv(Sigma_mat)
    R_effective = np.max(nla.eigvals(next_gen))
    R_effective_vec[r] = R_effective
    q_mat [r, :] = q
    rho_mat [r, :] = rho_im

    
    
    plt.figure(1)

    plt.plot(time_vec[:showplot_rho], q_mat[r, :showplot_rho], lw=2, label = str(np.round(RV, 2)))


plt.figure(1)
plt.grid(alpha = 0.5)
plt.title("Probability of no cases" )
plt.xlabel("Time (Days)")
plt.ylabel("Probability")
plt.ylim([0, 1])
plt.legend(loc=2, bbox_to_anchor=(1.05, 1), framealpha=1., edgecolor='black', fancybox=False, title = "R0")
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (8,4))
ax[0].grid(alpha = 0.5)
ax[1].grid(alpha = 0.5)
showplot_rho = 150
for r, beta_baseline in enumerate(beta_vec):
    # plt.figure(1)

    ax[0].plot(time_vec[:showplot_rho], q_mat[r, :showplot_rho], lw=2, label = str(np.round(R_effective_vec[r], 2)))
    ax[1].plot(time_vec[:showplot_rho], rho_mat[r, :showplot_rho], lw=2, label = str(np.round(R_effective_vec[r], 2)))

ax[0].set_title('Probability of Extinction')
plt.figure(1)
ax[1].set_title("Probability of no cases" )
ax[0].set_xlabel("Time (Days)")
ax[1].set_xlabel("Time (Days)")
ax[0].set_ylabel("Probability")
# ax[1].set_ylabel("Probability")
ax[1].label_outer()
ax[0].set_ylim([0, 1])
ax[1].set_ylim([0, 1])
# ax.label_outer()
plt.legend(loc=2, bbox_to_anchor=(1.05, 1), framealpha=1., edgecolor='black', fancybox=False, title = r"$R_{eff}$")
plt.tight_layout()
plt.savefig('./Figures/Figure_1.png')

In [ ]:

plt.figure()
plt.grid(alpha = 0.5)
plt.title("Value of T* " + r'$\left(\varepsilon = 10^{-3}\right)$')
plt.ylabel(r'$T^*$' + ' (Days after VoC Introduction)')


Istars_min = np.zeros(len(Reff_vec))
for r in range(0, len(Reff_vec)):
    sig_over_mean = (np.sqrt(variance[r, :])/(mean[r, :]))
    Tstar = mt.Tstar(time_vec, q_mat[r, :], sig_over_mean, thresh1 = epsilon, thresh2 = epsilon)

    Tstar_idxs[r] = int(Tstar[0])
    Tstar_idxs = Tstar_idxs.astype(int)
    Tstars[r] = Tstar[1]
    
    Istars_min[r] = mean[r, Tstar_idxs[r]]
plt.plot(R_effective_vec, Tstars)
plt.ylim([0, 200])
plt.xlabel(r'$R_{eff}$')
plt.tight_layout()
plt.savefig('./Figures/tstar_choice_im=0.png')

In [ ]:
save = 0

showplot = 300
Istars_min = np.zeros(len(Reff_vec))
for r in range(0, len(Reff_vec)):
    Istars_min[r] = mean[r, Tstar_idxs[r]]
    
showplot = 250

fig, axs = plt.subplots(2, 3, figsize = (8, 6))
for r in range(0, len(Reff_vec)):
    sig_vec = np.sqrt(variance)
    RV = np.round(Reff_vec[r], 2)
    if r<=2:
        b = 0
    else:
        b=1    
    color = 'tab:blue'
    if r ==0:
        
        axs[b, r%3].plot(time_vec[:showplot], q_mat[r, :showplot] , color = color, label = "P(Extinction)")
    else:
        axs[b, r%3].plot(time_vec[:showplot], q_mat[r, :showplot] , color = color)
        axs[b, r%3].axvline(time_vec[Tstar_idxs[r]], color = 'purple', linestyle = '--')
    axs[b, r%3].tick_params(axis='y', labelcolor=color)
    axs[b, r%3].grid(alpha = 0.5)
    axs[b, r%3].set_ylabel("Probability", color = color)
    axs[b, r%3].set_ylabel("Probability", color = color)
    axs[b, r%3].set_xlabel("Days")
    axs[b, r%3].set_ylim([0, 1])
    # axs[b, r%3].set_yscale("log")
    axs[b, r%3].set_title(r"$R_{eff} = $" + str(np.round(R_effective_vec[r], 2)))
    axs[-1, -1].set_visible(False)
    color= 'tab:orange'
    ax1 = axs[b, r%3].twinx()
    if r ==0:
        ax1.plot(time_vec[:showplot], (sig_vec[r, :showplot])/(mean[r, :showplot] ), color = color, label = "Coeff. of variation")
        ax1.set_ylim([0, 40])
        axs[b, r%3].axvline(time_vec[Tstar_idxs[r]], color = 'purple', linestyle = '--', label = str("T*"))

    else:
        ax1.plot(time_vec[:showplot], (sig_vec[r, :showplot])/(mean[r, :showplot] ), color = color)
        ax1.set_ylim([0, 40])
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_ylabel("sd/mean", color = color)
    
    
    
bottom_right_ax = axs[-1][-1] 
bottom_right_ax.clear()  
bottom_right_ax.set_axis_off()
plt.suptitle("Extinction Probability and Coefficient of Variation over time")
fig.legend(loc='lower right', bbox_to_anchor=(0.95, 0.2),framealpha=1., edgecolor='black', fancybox=False)
fig.tight_layout()
plt.savefig("./Figures/sig_over_mean_plot_im=0.png")


In [ ]:
plt.figure()
plt.grid(alpha = 0.5)
plt.title("Value of T* " + r'$\left(\varepsilon = 10^{-3}\right)$')

plt.ylabel(r'$T^*$' + ' (Days after VoC Introduction)')

Istars_min = np.zeros(len(Reff_vec))
for r in range(0, len(Reff_vec)):
    sig_over_mean = (np.sqrt(variance_im[r, :])/(mean_im[r, :]))
    Tstar = mt.Tstar(time_vec, rho_mat[r, :], sig_over_mean, thresh1 = epsilon, thresh2 = epsilon)
    
    Tstar_idxs[r] = int(Tstar[0])
    Tstar_idxs = Tstar_idxs.astype(int)
    Tstars[r] = Tstar[1]
    
    Istars_min[r] = mean[r, Tstar_idxs[r]]
plt.plot(R_effective_vec, Tstars)
plt.ylim([0, 200])
plt.xlabel(r'$R_{eff}$')

plt.tight_layout()
plt.savefig('./Figures/tstar_choice_im=0.1.png')

In [ ]:
save = 0

showplot = 300
Istars_min = np.zeros(len(Reff_vec))
for r in range(0, len(Reff_vec)):
    Istars_min[r] = mean[r, Tstar_idxs[r]]
    
#     tst_idx[r] = np.min(np.where(mean[r, :]>=threshold))
showplot = 250

fig, axs = plt.subplots(2, 3, figsize = (8, 6))
for r in range(0, len(Reff_vec)):
    sig_vec = np.sqrt(variance_im)
    
    if r<=2:
        b = 0
    else:
        b=1    
    color = 'tab:blue'
    if r ==0:
        
        axs[b, r%3].plot(time_vec[:showplot], rho_mat[r, :showplot] , color = color, label = "P(No cases)")
    else:
        axs[b, r%3].plot(time_vec[:showplot], rho_mat[r, :showplot] , color = color)
        axs[b, r%3].axvline(time_vec[Tstar_idxs[r]], color = 'purple', linestyle = '--')
    axs[b, r%3].tick_params(axis='y', labelcolor=color)
    axs[b, r%3].grid(alpha = 0.5)
    axs[b, r%3].set_ylabel("Probability", color = color)
    axs[b, r%3].set_ylabel("Probability", color = color)
    axs[b, r%3].set_xlabel("Days")
    axs[b, r%3].set_ylim([0, 1])
    # axs[b, r%3].set_yscale("log")
    axs[b, r%3].set_title(r"$R_{eff} = $" + str(np.round(R_effective_vec[r], 2)))
    axs[-1, -1].set_visible(False)
    color= 'tab:orange'
    ax1 = axs[b, r%3].twinx()
    if r ==0:
        ax1.plot(time_vec[:showplot], (sig_vec[r, :showplot])/(mean_im[r, :showplot]  ),
         color = color, label = "Coeff. of variation")
        ax1.set_ylim([0, 10])
        axs[b, r%3].axvline(time_vec[Tstar_idxs[r]], color = 'purple', linestyle = '--', label = str("T*"))

    else:
        ax1.plot(time_vec[:showplot], (sig_vec[r, :showplot])/(mean_im[r, :showplot] ),
         color = color)
        ax1.set_ylim([0, 10])
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_ylabel("sd/mean", color = color)
    
    
    
bottom_right_ax = axs[-1][-1] 
bottom_right_ax.clear()  
bottom_right_ax.set_axis_off()
plt.suptitle("Probability of zero cases and Coefficient of Variation over time")
fig.legend(loc='lower right', bbox_to_anchor=(0.95, 0.2),framealpha=1., edgecolor='black', fancybox=False)
fig.tight_layout()
# if save:
plt.savefig("Figures/sig_over_mean_plot_im=0.1.png")

In [ ]:
im_vec1 = np.array(im_vec)
im_vec2 = np.array(im_vec)
im_vec3 = np.array(im_vec)
im_vec4 = np.array(im_vec)
im_vec5 = np.array(im_vec)
im_vec1[0] = 0.1
im_vec2[0] = 0.2
im_vec3[0] = 0.05
im_vec4[0] = 0.5
im_vec5[0] = 0.075

In [ ]:
n_betas = len(beta_vec)
rho_mat_im_1 = np.zeros((n_betas, len(time_vec)))
rho_mat_im_2 = np.zeros((n_betas, len(time_vec)))
rho_mat_im_3 = np.zeros((n_betas, len(time_vec)))
rho_mat_im_4 = np.zeros((n_betas, len(time_vec)))
rho_mat_im_5 = np.zeros((n_betas, len(time_vec)))

# Initialise Mean

mean_im1 = np.zeros((n_betas, len(time_vec)))
mean_im2 = np.zeros((n_betas, len(time_vec)))
mean_im3 = np.zeros((n_betas, len(time_vec)))
mean_im4 = np.zeros((n_betas, len(time_vec)))
mean_im5 = np.zeros((n_betas, len(time_vec)))


    
# Initialise Variance
variance_im1 = np.zeros((n_betas, len(time_vec)))
variance_im2 = np.zeros((n_betas, len(time_vec)))
variance_im3 = np.zeros((n_betas, len(time_vec)))
variance_im4 = np.zeros((n_betas, len(time_vec)))
variance_im5 = np.zeros((n_betas, len(time_vec)))


R_effective_vec = np.zeros(n_betas)
for r, beta_baseline in enumerate(beta_vec):
    # beta_baseline = Reff*gamma

    RV = beta_baseline / gamma
    
    omega_vec = sigma * np.ones(ntypes) 

    
    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta_baseline)*trans_vec[nex] + gamma
        

    def P(u, t): # Offspring distribution

        pvec = np.zeros_like(u)
        nexposed = int(ntypes/2)
        omega = (np.sum(prop_vec * const_vec * beta_baseline) + gamma)
        

        sum_gen_funcs = np.sum((beta_baseline*prop_vec/omega)*const_vec*u[:nexposed])
         
        for nex in range(0, nexposed):

            pvec[nex] = u[nex+nexposed] 
            pvec[nex+nexposed] =  u[nex+nexposed] * sum_gen_funcs  + (gamma/omega)
        
        return pvec            
            
            
    eta = im_vec.astype(float)
    

# Mean of the Process
# Matrix of first partial derivatives of offspring distribution minus lifetimes*identity
    Omat = np.zeros((ntypes, ntypes))
    omega_vec = sigma * np.ones(ntypes)       

    for nex in range(0, nexposed):
        omega_vec[nex + nexposed] = np.sum(const_vec*prop_vec*beta_baseline)*trans_vec[nex] + gamma
        Omat[nex, nex+nexposed] = omega_vec[nex]
        Omat[nex+nexposed, :nexposed] = const_vec*prop_vec*beta_baseline *trans_vec[nex]
        Omat[nex+nexposed, nex+nexposed] = np.sum(const_vec * beta_baseline * prop_vec)*trans_vec[nex]

    Omat -= np.diag(omega_vec)


    

    Mvec = odeint(mt.set_mean_odes, y0, time_vec, args = (Omat,)).T
    # mean_arr[r, :, :] = Mvec

    # mean[r, :] = np.sum(Mvec, axis=0)

    Mvec_im1 = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec1)).T
    mean_im1[r, :] = np.sum(Mvec_im1, axis=0)
    Mvec_im2 = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec2)).T
    mean_im2[r, :] = np.sum(Mvec_im2, axis=0)
    Mvec_im3 = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec3)).T
    mean_im3[r, :] = np.sum(Mvec_im3, axis=0)
    Mvec_im4 = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec4)).T
    mean_im4[r, :] = np.sum(Mvec_im4, axis=0)
    Mvec_im5 = odeint(mt.set_mean_odes_im, y0_im, time_vec, args = (Omat, im_vec5)).T
    mean_im5[r, :] = np.sum(Mvec_im5, axis=0)

    
    rho_im1 = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec1)).T )[-1, :]
    rho_im2 = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec2)).T )[-1, :]
    rho_im3 = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec3)).T )[-1, :]
    rho_im4 = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec4)).T )[-1, :]
    rho_im5 = (odeint(mt.set_odes_im, ics_im, time_vec, args = (P, omega_vec, im_vec5)).T )[-1, :]
    rho_mat_im_1[r, :] = rho_im1
    rho_mat_im_2[r, :] = rho_im2
    rho_mat_im_3[r, :] = rho_im3
    rho_mat_im_4[r, :] = rho_im4
    rho_mat_im_5[r, :] = rho_im5
        
    # Variance and Coefficient of Variation
    param_dict = {"a":scale[0], "b":scale[1], "c":scale[2], "d":scale[3], "s":sigma, "g":gamma, "beta":beta_baseline}

    variance_im1[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec1)
    variance_im2[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec2)
    variance_im3[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec3)
    variance_im4[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec4)
    variance_im5[r, :] = mt.variance(time_vec, y0_im, omega_vec, Omat, [prop_vec, const_vec, sigma, beta_baseline, gamma], im = im_vec5)


In [ ]:
plt.figure()
plt.grid(alpha = 0.5)
plt.title("Value of T* " + r'$\left(\varepsilon = 10^{-3}\right)$')

R_effective_vec = np.array((1.32, 1.47, 1.71, 1.86, 2.15))
plt.ylabel(r'$T^*$' + ' (Days after VoC Introduction)')

Tstars1 = np.zeros(len(Reff_vec))
Tstars2 = np.zeros(len(Reff_vec))
Tstars3 = np.zeros(len(Reff_vec))
Tstars4 = np.zeros(len(Reff_vec))
Tstars5 = np.zeros(len(Reff_vec))

for r in range(0, len(Reff_vec)):
    sig_over_mean1 = (np.sqrt(variance_im1[r, :])/(mean_im1[r, :]) + np.finfo(float).eps)
    sig_over_mean2 = (np.sqrt(variance_im2[r, :])/(mean_im2[r, :]) + np.finfo(float).eps)
    sig_over_mean3 = (np.sqrt(variance_im3[r, :])/(mean_im3[r, :]) + np.finfo(float).eps)
    sig_over_mean4 = (np.sqrt(variance_im4[r, :])/(mean_im4[r, :]) + np.finfo(float).eps)
    sig_over_mean5 = (np.sqrt(variance_im5[r, :])/(mean_im5[r, :]) + np.finfo(float).eps)

    Tstar1 = mt.Tstar(time_vec, rho_mat_im_1[r, :], sig_over_mean1, thresh1 = epsilon, thresh2 = epsilon)
    Tstar2 = mt.Tstar(time_vec, rho_mat_im_2[r, :], sig_over_mean2, thresh1 = epsilon, thresh2 = epsilon)
    Tstar3 = mt.Tstar(time_vec, rho_mat_im_3[r, :], sig_over_mean3, thresh1 = epsilon, thresh2 = epsilon)
    Tstar4 = mt.Tstar(time_vec, rho_mat_im_4[r, :], sig_over_mean4, thresh1 = epsilon, thresh2 = epsilon)
    Tstar5 = mt.Tstar(time_vec, rho_mat_im_5[r, :], sig_over_mean5, thresh1 = epsilon, thresh2 = epsilon)


    Tstars1[r] = Tstar1[1]
    Tstars2[r] = Tstar2[1]
    Tstars3[r] = Tstar3[1]
    Tstars4[r] = Tstar4[1]
    Tstars5[r] = Tstar5[1]
    
plt.plot(R_effective_vec, Tstars3, label = '0.05', color = cmp(1), linewidth = 1.5)
plt.plot(R_effective_vec, Tstars5, label = '0.075', color = cmp(2), linewidth = 1.5)
plt.plot(R_effective_vec, Tstars1, label = '0.1', color = cmp(0), linewidth = 3)
plt.plot(R_effective_vec, Tstars2, label = '0.2', color = cmp(3), linewidth = 1.5)
plt.plot(R_effective_vec, Tstars4, label = '0.5', color=cmp(4), linewidth = 1.5)

plt.ylim([0, 200])
plt.xlabel(r'$R_{eff}$')
plt.legend(title = r'$\eta$')
plt.tight_layout()
plt.savefig('Figures/tstar_choice_im_many.eps')